In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
import time
import sys
import os
from multiprocessing import Pool
from functools import partial
directory = 'figures'
if not os.path.exists(directory):
    os.makedirs(directory)
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.notebook import tqdm
import numpy as np
from dask.distributed import Client, progress, LocalCluster
from dask import compute, delayed
import dask.array as da
from dask.diagnostics import ProgressBar
plt.rcParams['figure.dpi']=400

In [ ]:
def denseH_scar(L, Omega, periodic=True):
    
    
    dim = 2 ** L # dimensions of the Hilbert space
    
    H = np.zeros((dim, dim)) # initliaze the Hamiltonian
    
    "Calculation of off-diagonal elements due to the magnetic field"
    
    for beta in range(dim): # iterate over all states
        
        for j in range(1,L+1): # iterate over all sites
            
            alpha = beta ^ (1<<j-1) # flips jth bit of beta to get the state alpha that is related to beta by a single bit flip
            
            H[alpha, beta] += Omega/2 # contribution by sigma^j_x
            
    "Calculation of diagonal elements due to Ising interaction"

    for alpha in range(dim): # iterate over all states
        
        for j in range(1, L-1): # iterate over all sites
            
            if 2*(alpha & (1 << j-1)) != alpha & (1 << j): # check if site j and j+1 dont have the same spin
                
                if alpha & (1 << j+1) == 0:
                    H[alpha, alpha] += 1/2 # if j+2 is spin up increase energy by 1/2, else decrease
                
                else:
                    H[alpha, alpha] -= 1/2
        
        if periodic:
            
            
            if 2*(alpha & (1 << L-2)) != (alpha & (1 << L-1)): # Check if the states at site L-1 and L have same spin

                    
                if alpha & (1 << 0) == 0:
                    H[alpha, alpha] += 1/2 # if j+2 is spin up increase energy by 1/2, else decrease
                else:
                    H[alpha, alpha] -= 1/2 
                    
            if (alpha & (1 << L-1)) != ((alpha & (1 << 0))*(2**(L-1))): # Check if the states at either end have the same spin
                
                if alpha & (1 << 1) == 0:
                    H[alpha, alpha] += 1/2 # if j+2 is spin up increase energy by 1/2, else decrease
                    
                else:
                    H[alpha, alpha] -= 1/2 


    return H

def diagonalize(Ls, Omega):
    
    eigss = []
    vecss = []
    
    for L in Ls:
        H = denseH_scar(L, Omega)
        eigs, vecs = scipy.linalg.eigh(H)
        eigss.append(eigs)
        vecss.append(vecs)
    
    return eigss, vecss

In [ ]:
Ls = [6, 8, 10, 12, 14]
Omega = np.pi


start_time = time.time()
eigss, vecss = diagonalize(Ls, Omega)
end_time = time.time()
print('time taken:', end_time-start_time)


In [ ]:
def entanglement_entropy(L, l, coeffs):
    coeffs_matrix = coeffs.reshape(2**l, 2**(L-l))
    s = np.linalg.svd(coeffs_matrix, compute_uv=False)
    lambdas = s**2
    S = -np.sum(lambdas * np.log(lambdas))
    return S

In [ ]:
entropies = {}
xkcd_colors = ['xkcd:royal blue', 'xkcd:green', 'xkcd:orange', 'xkcd:hot pink', 'xkcd:indigo']

for i, L in enumerate(Ls):
    entropies_L = []
    for vec in tqdm(vecss[i].T, desc=f"Computing entanglement entropy for L={L}"):
        S = entanglement_entropy(L, L//2, vec)
        entropies_L.append(S)
    entropies[L] = entropies_L

plt.figure(figsize=(12, 8))

for i, L in enumerate(Ls):
    energies = eigss[i]
    entropies_L = np.array(entropies[L])
    plt.scatter(energies, entropies_L, label=f'L={L}', color=xkcd_colors[i])

    # Calculate and plot vertical lines for the scar states
for i, m in enumerate(range(-L//2, L//2 + 1)):
    scar_energy = Omega * m
    
    if i == 0:
        plt.axvline(x=scar_energy, color='grey', linestyle='dashdot', linewidth=1, label=f'Expected scar states')
    else:
        plt.axvline(x=scar_energy, color='grey', linestyle='dashdot', linewidth=1)

plt.xlabel('$\epsilon_n$')
plt.ylabel('$S_{L/2}$')
plt.legend()
plt.title('Entanglement Entropy vs. Energy Density')
plt.savefig('entropy_Scar.png', dpi=400)
plt.show()

In [ ]:
a=[1, 2, 3, 4]

In [ ]:
for i, L in enumerate(a[1:]):
    print(i)
    print(L)


In [ ]:
from scipy.optimize import curve_fit

scar_entropies = []
L_values = []
for i, L in enumerate(Ls):
    eigs = eigss[i]  # Eigenvalues for the system size L
    vecs = vecss[i]  # Eigenvectors for the system size L
    m = 0
    scar_energy = Omega * m  # Energy of the scar state with m = 0
    index = np.argmin(np.abs(eigs - scar_energy))  # Find the index of the scar state
    vec = vecs[:, index]
    S = entanglement_entropy(L, L//2, vec)
    scar_entropies.append(S)
    L_values.append(L)

L_values = np.array(L_values)
scar_entropies = np.array(scar_entropies)

coefficients = np.polyfit(L_values, scar_entropies, 1)
linear_fit = np.poly1d(coefficients)

plt.figure(figsize=(10, 6))
plt.scatter(L_values, scar_entropies, color='red', label='Scar State Entropies')
plt.plot(L_values, linear_fit(L_values), label=f'Linear Fit: $S = {coefficients[0]:.2f}L + {coefficients[1]:.2f}$')
plt.xlabel('System Size $L$')
plt.ylabel('Entanglement Entropy $S_{L/2}$')
plt.legend()
plt.title('Scaling of Entanglement Entropy for Scar States')
plt.savefig('entropy-fit.png', dpi = 400)
plt.show()

In [ ]:
from scipy.optimize import curve_fit

scar_entropies = []
L_values = []
for i, L in enumerate(Ls):
    eigs = eigss[i]  # Eigenvalues for the system size L
    vecs = vecss[i]  # Eigenvectors for the system size L
    m = 2
    scar_energy = Omega * m  # Energy of the scar state with m = 0
    index = np.argmin(np.abs(eigs - scar_energy))  # Find the index of the scar state
    vec = vecs[:, index]
    S = entanglement_entropy(L, L//2, vec)
    scar_entropies.append(S)
    L_values.append(L)

L_values = np.array(L_values)
scar_entropies = np.array(scar_entropies)

# Perform a logarithmic fit
log_L_values = np.log(L_values)
coefficients = np.polyfit(log_L_values, scar_entropies, 1)
log_fit = np.poly1d(coefficients)

plt.figure(figsize=(10, 6))
plt.scatter(L_values, scar_entropies, color='red', label='Scar State Entropies')
plt.plot(L_values, log_fit(log_L_values), label=f'Logarithmic Fit: $S = {coefficients[0]:.2f} \log(L) + {coefficients[1]:.2f}$')
plt.xlabel('System Size $L$')
plt.ylabel('Entanglement Entropy $S_{L/2}$')
plt.legend()
plt.title('Scaling of Entanglement Entropy for Scar States')
plt.show()